In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#전처리
from sklearn.preprocessing import LabelEncoder, StandardScaler

#데이터분리
from sklearn.model_selection import train_test_split

#모델링
import lightgbm as lgbm

#성과지표
from sklearn.metrics import accuracy_score

In [2]:
#data load
train  = pd.read_csv('./train.csv', index_col=0)
test = pd.read_csv('./test_x.csv', index_col=0)
submission=pd.read_csv('./sample_submission.csv', index_col=0)

print(train.shape)
print(test.shape)
print(submission.shape)

(45532, 77)
(11383, 76)
(11383, 1)


In [3]:
#labelencoding : gender, age_group, race, religion 전처리
label = LabelEncoder()
train['gender_code'] = label.fit_transform(train['gender'])
train['age_group_code'] = label.fit_transform(train['age_group'])
train['race_code'] = label.fit_transform(train['race'])
train['religion_code'] = label.fit_transform(train['religion'])

test['gender_code'] = label.fit_transform(test['gender'])
test['age_group_code'] = label.fit_transform(test['age_group'])
test['race_code'] = label.fit_transform(test['race'])
test['religion_code'] = label.fit_transform(test['religion'])

drop2 = ['gender','age_group','race','religion']
train = train.drop(drop2, axis=1)
test = test.drop(drop2, axis=1)

In [4]:
#데이터프레임 시각화
pd.set_option('display.max_columns', 77)
pd.set_option('display.max_rows', 100)
train.head(20)

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,education,engnat,familysize,hand,married,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,voted,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,gender_code,age_group_code,race_code,religion_code
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3.0,363,4.0,1370,5.0,997,1.0,1024,2.0,1577,5.0,539,2.0,586,4.0,1095,5.0,1142,4.0,1287,4.0,883,4.0,851,2.0,851,5.0,816,2.0,579,2.0,924,2.0,366,2.0,876,2.0,633,1.0,1115,2,1,4,1,3,2,2,2,1,2,1,7,4,4,3,1,2,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,1,0,3,6,10
1,5.0,647,5.0,1313,3.0,3387,5.0,2969,1.0,4320,3.0,2190,1.0,826,1.0,4082,5.0,1867,3.0,1264,5.0,2943,4.0,3927,1.0,4329,5.0,1828,1.0,1214,5.0,2414,5.0,1356,1.0,3039,4.0,4304,1.0,1346,4,2,3,1,1,1,1,0,0,1,2,3,4,0,4,3,2,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,1,0,2,1,7
2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,1.0,531,4.0,1167,1.0,1016,3.0,2653,2.0,1569,5.0,998,5.0,2547,2.0,918,4.0,2153,2.0,1304,1.0,1131,5.0,937,4.0,1327,1.0,1170,1.0,1409,3,1,3,1,2,2,3,1,5,3,4,2,6,1,3,2,1,0,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,1,3,6,10
3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,2.0,1519,4.0,159,3.0,2275,5.0,2809,4.0,5614,3.0,3219,4.0,1296,4.0,9046,4.0,1216,4.0,1169,4.0,23868,3.0,581,4.0,8830,4.0,2392,5.0,1312,4,2,0,1,1,2,4,1,1,1,3,1,3,1,3,3,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,0,2,1,7
4,1.0,927,1.0,707,5.0,556,2.0,1062,1.0,1014,2.0,628,1.0,991,1.0,1259,5.0,1153,5.0,1388,5.0,740,5.0,1181,4.0,547,2.0,575,1.0,754,4.0,1140,5.0,323,5.0,1070,1.0,583,2.0,1889,3,1,2,1,2,1,1,1,6,0,2,0,6,2,6,1,1,0,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,2,6,0
5,2.0,834,1.0,1769,4.0,2105,1.0,1070,5.0,1930,2.0,1258,5.0,1451,1.0,1029,3.0,1633,1.0,2254,5.0,1984,1.0,1158,3.0,1194,3.0,483,1.0,983,1.0,817,5.0,1052,1.0,1354,1.0,747,2.0,1826,2,1,6,1,3,5,5,5,4,5,3,3,7,5,3,2,2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,4,6,10
6,1.0,1382,1.0,1473,5.0,1479,4.0,2403,1.0,616,2.0,549,5.0,390,4.0,1601,5.0,1312,1.0,1699,4.0,1229,4.0,1175,1.0,1078,4.0,341,5.0,635,2.0,1448,4.0,838,4.0,765,3.0,1954,1.0,1482,2,1,3,1,1,2,4,3,5,1,1,3,2,1,5,2,1,0,1,0,1,1,1,1,1,1,1,1,0,1,0,1,1,1,2,6,1
7,1.0,384,1.0,908,5.0,870,1.0,1059,1.0,1266,1.0,483,1.0,592,1.0,2020,5.0,6022,5.0,2090,4.0,1767,5.0,982,5.0,1576,1.0,782,3.0,911,5.0,417,1.0,392,5.0,1510,1.0,842,1.0,1138,2,1,1,1,1,3,6,0,0,0,0,0,1,2,6,2,1,1,0,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,3,6,5
8,5.0,795,2.0,3469,4.0,1693,3.0,1991,4.0,5922,2.0,1322,5.0,1014,5.0,3014,3.0,3771,3.0,13295,5.0,3605,3.0,9265,4.0,2531,1.0,812,4.0,2595,5.0,1227,5.0,1837,4.0,1218,4.0,9923,5.0,1631,4,2,0,2,1,1,1,2,1,2,2,2,2,2,3,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,2,5,5


In [5]:
# drop_val1 변수 제거
drop_val1 = ['QaA', 'QaE', 'QbE', 'QcE', 'QdA', 'QdE', 'QeA', 'QeE','QfE', 'QgA', 'QgE', 'QhA', 'QhE', 'QiA', 'QiE','QjE',
       'QkA', 'QkE', 'QlA', 'QlE', 'QmE', 'QnA', 'QnE', 'QoE','QpA', 'QpE', 'QqE', 'QrA', 'QrE', 'QsE', 'QtA', 'QtE',
        'engnat', 'familysize', 'hand','tp01','tp03', 'tp04', 'tp05','wf_01','wf_02','wf_03','tp07', 'tp10', 'gender_code']
#남겨둔 변수 : QbA, QcA, QfA,QjA ,QmA, QoA, QqA,QsA, 'education', 'married','age_group_code', 'race_code', 'religion_code'
#남겨둔 변수 :'wr_01', 'wr_02', 'wr_03', 'wr_04','wr_05', 'wr_06', 'wr_07', 'wr_08', 'wr_09', 'wr_10', 'wr_11', 'wr_12','wr_13'
#남겨둔 변수 : 'urban','tp02', tp06, tp08, tp09
train = train.drop (drop_val1, axis=1)
test = test.drop (drop_val1, axis=1)
train.head()


,QbA,QcA,QfA,QjA,QmA,QoA,QqA,QsA,education,married,tp02,tp06,tp08,tp09,urban,voted,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,age_group_code,race_code,religion_code
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4.0,5.0,5.0,4.0,2.0,2.0,2.0,2.0,2,3,2,1,4,4,1,2,0,1,0,0,1,0,1,1,0,1,0,1,1,3,6,10
1,5.0,3.0,3.0,3.0,1.0,1.0,5.0,4.0,4,1,1,2,4,0,3,2,0,1,0,1,1,0,1,1,0,1,0,1,1,2,1,7
2,1.0,1.0,1.0,2.0,2.0,2.0,5.0,1.0,3,2,3,4,6,1,2,1,1,1,0,1,1,0,1,1,1,1,0,1,1,3,6,10
3,3.0,4.0,2.0,4.0,4.0,4.0,3.0,4.0,4,1,4,3,3,1,3,1,0,1,0,0,0,0,0,1,0,1,0,1,1,2,1,7
4,1.0,5.0,2.0,5.0,4.0,1.0,5.0,1.0,3,2,1,2,6,2,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,2,6,0


In [9]:
#wr_(01~13) 총합 계산해서 wr_total 열에 저장하기
train['wr_total']=0
test['wr_total']=0
wr_list = ['wr_01', 'wr_02', 'wr_03', 'wr_04','wr_05', 'wr_06', 'wr_07', 'wr_08', 'wr_09', 'wr_10', 'wr_11', 'wr_12','wr_13']

print('train 데이터 wr_total 계산')
for index in range(len(train)):
    for wr in wr_list:
        if(train.loc[index][wr]==1):
            train.loc[index, 'wr_total']+=1
    print( index+1,'번째 wr합',train.loc[index]['wr_total'])
print('\n')

print('test 데이터 wr_total  계산')
for index in range(len(test)):
    for wr in wr_list:
        if(test.loc[index][wr]==1):
            test.loc[index, 'wr_total']+=1
    print( index+1,'번째 wr합',test.loc[index]['wr_total'])

train 데이터 wr_total 계산
1 번째 wr합 7.0
2 번째 wr합 8.0
3 번째 wr합 10.0
4 번째 wr합 5.0
5 번째 wr합 11.0
6 번째 wr합 2.0
7 번째 wr합 11.0
8 번째 wr합 11.0
9 번째 wr합 3.0
10 번째 wr합 10.0
11 번째 wr합 8.0
12 번째 wr합 8.0
13 번째 wr합 10.0
14 번째 wr합 8.0
15 번째 wr합 8.0
16 번째 wr합 10.0
17 번째 wr합 9.0
18 번째 wr합 10.0
19 번째 wr합 12.0
20 번째 wr합 10.0
21 번째 wr합 9.0
22 번째 wr합 10.0
23 번째 wr합 7.0
24 번째 wr합 9.0
25 번째 wr합 10.0
26 번째 wr합 9.0
27 번째 wr합 9.0
28 번째 wr합 10.0
29 번째 wr합 10.0
30 번째 wr합 11.0
31 번째 wr합 11.0
32 번째 wr합 9.0
33 번째 wr합 9.0
34 번째 wr합 10.0
35 번째 wr합 9.0
36 번째 wr합 12.0
37 번째 wr합 8.0
38 번째 wr합 8.0
39 번째 wr합 12.0
40 번째 wr합 8.0
41 번째 wr합 4.0
42 번째 wr합 9.0
43 번째 wr합 11.0
44 번째 wr합 10.0
45 번째 wr합 7.0
46 번째 wr합 9.0
47 번째 wr합 10.0
48 번째 wr합 8.0
49 번째 wr합 10.0
50 번째 wr합 8.0
51 번째 wr합 13.0
52 번째 wr합 11.0
53 번째 wr합 7.0
54 번째 wr합 10.0
55 번째 wr합 10.0
56 번째 wr합 9.0
57 번째 wr합 11.0
58 번째 wr합 12.0
59 번째 wr합 11.0
60 번째 wr합 8.0
61 번째 wr합 8.0
62 번째 wr합 10.0
63 번째 wr합 6.0
64 번째 wr합 13.0
65 번째 wr합 8.0
66 번째 wr합 9.0
67 번째 wr합 11.0
68 번째 wr합 1.0
69

559 번째 wr합 10.0
560 번째 wr합 5.0
561 번째 wr합 11.0
562 번째 wr합 10.0
563 번째 wr합 0.0
564 번째 wr합 10.0
565 번째 wr합 11.0
566 번째 wr합 9.0
567 번째 wr합 12.0
568 번째 wr합 9.0
569 번째 wr합 13.0
570 번째 wr합 11.0
571 번째 wr합 11.0
572 번째 wr합 8.0
573 번째 wr합 9.0
574 번째 wr합 8.0
575 번째 wr합 10.0
576 번째 wr합 10.0
577 번째 wr합 13.0
578 번째 wr합 10.0
579 번째 wr합 12.0
580 번째 wr합 10.0
581 번째 wr합 10.0
582 번째 wr합 8.0
583 번째 wr합 10.0
584 번째 wr합 10.0
585 번째 wr합 5.0
586 번째 wr합 8.0
587 번째 wr합 9.0
588 번째 wr합 8.0
589 번째 wr합 10.0
590 번째 wr합 12.0
591 번째 wr합 7.0
592 번째 wr합 7.0
593 번째 wr합 10.0
594 번째 wr합 10.0
595 번째 wr합 10.0
596 번째 wr합 10.0
597 번째 wr합 9.0
598 번째 wr합 12.0
599 번째 wr합 10.0
600 번째 wr합 12.0
601 번째 wr합 9.0
602 번째 wr합 0.0
603 번째 wr합 12.0
604 번째 wr합 8.0
605 번째 wr합 9.0
606 번째 wr합 9.0
607 번째 wr합 11.0
608 번째 wr합 9.0
609 번째 wr합 13.0
610 번째 wr합 13.0
611 번째 wr합 9.0
612 번째 wr합 13.0
613 번째 wr합 9.0
614 번째 wr합 9.0
615 번째 wr합 9.0
616 번째 wr합 9.0
617 번째 wr합 8.0
618 번째 wr합 9.0
619 번째 wr합 10.0
620 번째 wr합 10.0
621 번째 wr합 13.0
622 번째 wr합 5.0
623 번

1084 번째 wr합 8.0
1085 번째 wr합 8.0
1086 번째 wr합 11.0
1087 번째 wr합 13.0
1088 번째 wr합 10.0
1089 번째 wr합 10.0
1090 번째 wr합 8.0
1091 번째 wr합 10.0
1092 번째 wr합 10.0
1093 번째 wr합 9.0
1094 번째 wr합 11.0
1095 번째 wr합 9.0
1096 번째 wr합 10.0
1097 번째 wr합 2.0
1098 번째 wr합 8.0
1099 번째 wr합 9.0
1100 번째 wr합 9.0
1101 번째 wr합 10.0
1102 번째 wr합 7.0
1103 번째 wr합 7.0
1104 번째 wr합 11.0
1105 번째 wr합 8.0
1106 번째 wr합 8.0
1107 번째 wr합 12.0
1108 번째 wr합 6.0
1109 번째 wr합 10.0
1110 번째 wr합 9.0
1111 번째 wr합 13.0
1112 번째 wr합 10.0
1113 번째 wr합 9.0
1114 번째 wr합 9.0
1115 번째 wr합 6.0
1116 번째 wr합 11.0
1117 번째 wr합 11.0
1118 번째 wr합 8.0
1119 번째 wr합 13.0
1120 번째 wr합 10.0
1121 번째 wr합 9.0
1122 번째 wr합 11.0
1123 번째 wr합 9.0
1124 번째 wr합 8.0
1125 번째 wr합 7.0
1126 번째 wr합 9.0
1127 번째 wr합 11.0
1128 번째 wr합 10.0
1129 번째 wr합 13.0
1130 번째 wr합 11.0
1131 번째 wr합 4.0
1132 번째 wr합 9.0
1133 번째 wr합 10.0
1134 번째 wr합 7.0
1135 번째 wr합 10.0
1136 번째 wr합 13.0
1137 번째 wr합 12.0
1138 번째 wr합 10.0
1139 번째 wr합 9.0
1140 번째 wr합 6.0
1141 번째 wr합 4.0
1142 번째 wr합 8.0
1143 번째 wr합 9.0
1144 번째 wr합 

1602 번째 wr합 5.0
1603 번째 wr합 9.0
1604 번째 wr합 12.0
1605 번째 wr합 9.0
1606 번째 wr합 9.0
1607 번째 wr합 11.0
1608 번째 wr합 9.0
1609 번째 wr합 7.0
1610 번째 wr합 8.0
1611 번째 wr합 13.0
1612 번째 wr합 13.0
1613 번째 wr합 10.0
1614 번째 wr합 1.0
1615 번째 wr합 8.0
1616 번째 wr합 12.0
1617 번째 wr합 8.0
1618 번째 wr합 9.0
1619 번째 wr합 10.0
1620 번째 wr합 11.0
1621 번째 wr합 13.0
1622 번째 wr합 10.0
1623 번째 wr합 13.0
1624 번째 wr합 13.0
1625 번째 wr합 12.0
1626 번째 wr합 8.0
1627 번째 wr합 9.0
1628 번째 wr합 8.0
1629 번째 wr합 12.0
1630 번째 wr합 9.0
1631 번째 wr합 10.0
1632 번째 wr합 9.0
1633 번째 wr합 11.0
1634 번째 wr합 7.0
1635 번째 wr합 8.0
1636 번째 wr합 7.0
1637 번째 wr합 11.0
1638 번째 wr합 9.0
1639 번째 wr합 7.0
1640 번째 wr합 10.0
1641 번째 wr합 7.0
1642 번째 wr합 8.0
1643 번째 wr합 6.0
1644 번째 wr합 11.0
1645 번째 wr합 8.0
1646 번째 wr합 12.0
1647 번째 wr합 11.0
1648 번째 wr합 10.0
1649 번째 wr합 7.0
1650 번째 wr합 11.0
1651 번째 wr합 8.0
1652 번째 wr합 10.0
1653 번째 wr합 9.0
1654 번째 wr합 9.0
1655 번째 wr합 12.0
1656 번째 wr합 7.0
1657 번째 wr합 5.0
1658 번째 wr합 12.0
1659 번째 wr합 7.0
1660 번째 wr합 5.0
1661 번째 wr합 9.0
1662 번째 wr합 9.

2120 번째 wr합 9.0
2121 번째 wr합 8.0
2122 번째 wr합 12.0
2123 번째 wr합 10.0
2124 번째 wr합 9.0
2125 번째 wr합 10.0
2126 번째 wr합 9.0
2127 번째 wr합 11.0
2128 번째 wr합 9.0
2129 번째 wr합 8.0
2130 번째 wr합 3.0
2131 번째 wr합 11.0
2132 번째 wr합 10.0
2133 번째 wr합 9.0
2134 번째 wr합 12.0
2135 번째 wr합 11.0
2136 번째 wr합 1.0
2137 번째 wr합 10.0
2138 번째 wr합 7.0
2139 번째 wr합 10.0
2140 번째 wr합 10.0
2141 번째 wr합 4.0
2142 번째 wr합 10.0
2143 번째 wr합 9.0
2144 번째 wr합 9.0
2145 번째 wr합 12.0
2146 번째 wr합 9.0
2147 번째 wr합 10.0
2148 번째 wr합 10.0
2149 번째 wr합 6.0
2150 번째 wr합 9.0
2151 번째 wr합 11.0
2152 번째 wr합 11.0
2153 번째 wr합 11.0
2154 번째 wr합 13.0
2155 번째 wr합 12.0
2156 번째 wr합 9.0
2157 번째 wr합 11.0
2158 번째 wr합 11.0
2159 번째 wr합 13.0
2160 번째 wr합 8.0
2161 번째 wr합 9.0
2162 번째 wr합 2.0
2163 번째 wr합 11.0
2164 번째 wr합 13.0
2165 번째 wr합 12.0
2166 번째 wr합 10.0
2167 번째 wr합 11.0
2168 번째 wr합 10.0
2169 번째 wr합 9.0
2170 번째 wr합 9.0
2171 번째 wr합 9.0
2172 번째 wr합 7.0
2173 번째 wr합 11.0
2174 번째 wr합 11.0
2175 번째 wr합 11.0
2176 번째 wr합 8.0
2177 번째 wr합 9.0
2178 번째 wr합 7.0
2179 번째 wr합 6.0
2180 번째 

...

9389 번째 wr합 12.0
9390 번째 wr합 10.0
9391 번째 wr합 8.0
9392 번째 wr합 9.0
9393 번째 wr합 5.0
9394 번째 wr합 8.0
9395 번째 wr합 7.0
9396 번째 wr합 5.0
9397 번째 wr합 6.0
9398 번째 wr합 7.0
9399 번째 wr합 10.0
9400 번째 wr합 9.0
9401 번째 wr합 10.0
9402 번째 wr합 11.0
9403 번째 wr합 13.0
9404 번째 wr합 13.0
9405 번째 wr합 5.0
9406 번째 wr합 8.0
9407 번째 wr합 4.0
9408 번째 wr합 11.0
9409 번째 wr합 7.0
9410 번째 wr합 6.0
9411 번째 wr합 9.0
9412 번째 wr합 10.0
9413 번째 wr합 9.0
9414 번째 wr합 9.0
9415 번째 wr합 9.0
9416 번째 wr합 9.0
9417 번째 wr합 11.0
9418 번째 wr합 9.0
9419 번째 wr합 9.0
9420 번째 wr합 9.0
9421 번째 wr합 9.0
9422 번째 wr합 11.0
9423 번째 wr합 9.0
9424 번째 wr합 11.0
9425 번째 wr합 9.0
9426 번째 wr합 9.0
9427 번째 wr합 9.0
9428 번째 wr합 11.0
9429 번째 wr합 10.0
9430 번째 wr합 8.0
9431 번째 wr합 7.0
9432 번째 wr합 6.0
9433 번째 wr합 11.0
9434 번째 wr합 9.0
9435 번째 wr합 10.0
9436 번째 wr합 9.0
9437 번째 wr합 5.0
9438 번째 wr합 11.0
9439 번째 wr합 9.0
9440 번째 wr합 7.0
9441 번째 wr합 11.0
9442 번째 wr합 10.0
9443 번째 wr합 8.0
9444 번째 wr합 9.0
9445 번째 wr합 10.0
9446 번째 wr합 7.0
9447 번째 wr합 9.0
9448 번째 wr합 10.0
9449 번째 wr합 12.0
94

9898 번째 wr합 9.0
9899 번째 wr합 12.0
9900 번째 wr합 5.0
9901 번째 wr합 13.0
9902 번째 wr합 9.0
9903 번째 wr합 6.0
9904 번째 wr합 7.0
9905 번째 wr합 11.0
9906 번째 wr합 10.0
9907 번째 wr합 9.0
9908 번째 wr합 8.0
9909 번째 wr합 9.0
9910 번째 wr합 10.0
9911 번째 wr합 8.0
9912 번째 wr합 5.0
9913 번째 wr합 9.0
9914 번째 wr합 11.0
9915 번째 wr합 9.0
9916 번째 wr합 11.0
9917 번째 wr합 11.0
9918 번째 wr합 9.0
9919 번째 wr합 9.0
9920 번째 wr합 9.0
9921 번째 wr합 9.0
9922 번째 wr합 9.0
9923 번째 wr합 9.0
9924 번째 wr합 12.0
9925 번째 wr합 7.0
9926 번째 wr합 10.0
9927 번째 wr합 6.0
9928 번째 wr합 10.0
9929 번째 wr합 13.0
9930 번째 wr합 10.0
9931 번째 wr합 12.0
9932 번째 wr합 9.0
9933 번째 wr합 8.0
9934 번째 wr합 11.0
9935 번째 wr합 8.0
9936 번째 wr합 10.0
9937 번째 wr합 9.0
9938 번째 wr합 7.0
9939 번째 wr합 10.0
9940 번째 wr합 10.0
9941 번째 wr합 10.0
9942 번째 wr합 10.0
9943 번째 wr합 12.0
9944 번째 wr합 12.0
9945 번째 wr합 9.0
9946 번째 wr합 11.0
9947 번째 wr합 10.0
9948 번째 wr합 9.0
9949 번째 wr합 8.0
9950 번째 wr합 11.0
9951 번째 wr합 11.0
9952 번째 wr합 10.0
9953 번째 wr합 9.0
9954 번째 wr합 9.0
9955 번째 wr합 8.0
9956 번째 wr합 10.0
9957 번째 wr합 12.0
9958 번째 wr합

10408 번째 wr합 6.0
10409 번째 wr합 9.0
10410 번째 wr합 12.0
10411 번째 wr합 2.0
10412 번째 wr합 1.0
10413 번째 wr합 9.0
10414 번째 wr합 9.0
10415 번째 wr합 6.0
10416 번째 wr합 8.0
10417 번째 wr합 9.0
10418 번째 wr합 10.0
10419 번째 wr합 11.0
10420 번째 wr합 9.0
10421 번째 wr합 9.0
10422 번째 wr합 9.0
10423 번째 wr합 7.0
10424 번째 wr합 2.0
10425 번째 wr합 5.0
10426 번째 wr합 10.0
10427 번째 wr합 9.0
10428 번째 wr합 12.0
10429 번째 wr합 9.0
10430 번째 wr합 9.0
10431 번째 wr합 10.0
10432 번째 wr합 12.0
10433 번째 wr합 10.0
10434 번째 wr합 9.0
10435 번째 wr합 13.0
10436 번째 wr합 8.0
10437 번째 wr합 9.0
10438 번째 wr합 9.0
10439 번째 wr합 13.0
10440 번째 wr합 10.0
10441 번째 wr합 9.0
10442 번째 wr합 8.0
10443 번째 wr합 9.0
10444 번째 wr합 9.0
10445 번째 wr합 10.0
10446 번째 wr합 12.0
10447 번째 wr합 10.0
10448 번째 wr합 10.0
10449 번째 wr합 10.0
10450 번째 wr합 8.0
10451 번째 wr합 6.0
10452 번째 wr합 7.0
10453 번째 wr합 8.0
10454 번째 wr합 12.0
10455 번째 wr합 13.0
10456 번째 wr합 13.0
10457 번째 wr합 9.0
10458 번째 wr합 12.0
10459 번째 wr합 9.0
10460 번째 wr합 10.0
10461 번째 wr합 9.0
10462 번째 wr합 4.0
10463 번째 wr합 13.0
10464 번째 wr합 5.0
10465 번째 

10890 번째 wr합 9.0
10891 번째 wr합 8.0
10892 번째 wr합 10.0
10893 번째 wr합 10.0
10894 번째 wr합 9.0
10895 번째 wr합 12.0
10896 번째 wr합 9.0
10897 번째 wr합 9.0
10898 번째 wr합 10.0
10899 번째 wr합 10.0
10900 번째 wr합 1.0
10901 번째 wr합 9.0
10902 번째 wr합 8.0
10903 번째 wr합 5.0
10904 번째 wr합 8.0
10905 번째 wr합 10.0
10906 번째 wr합 10.0
10907 번째 wr합 8.0
10908 번째 wr합 4.0
10909 번째 wr합 10.0
10910 번째 wr합 13.0
10911 번째 wr합 6.0
10912 번째 wr합 6.0
10913 번째 wr합 10.0
10914 번째 wr합 7.0
10915 번째 wr합 10.0
10916 번째 wr합 11.0
10917 번째 wr합 6.0
10918 번째 wr합 7.0
10919 번째 wr합 6.0
10920 번째 wr합 11.0
10921 번째 wr합 0.0
10922 번째 wr합 6.0
10923 번째 wr합 8.0
10924 번째 wr합 8.0
10925 번째 wr합 11.0
10926 번째 wr합 10.0
10927 번째 wr합 9.0
10928 번째 wr합 5.0
10929 번째 wr합 8.0
10930 번째 wr합 9.0
10931 번째 wr합 8.0
10932 번째 wr합 8.0
10933 번째 wr합 9.0
10934 번째 wr합 9.0
10935 번째 wr합 12.0
10936 번째 wr합 11.0
10937 번째 wr합 11.0
10938 번째 wr합 11.0
10939 번째 wr합 13.0
10940 번째 wr합 10.0
10941 번째 wr합 12.0
10942 번째 wr합 10.0
10943 번째 wr합 10.0
10944 번째 wr합 10.0
10945 번째 wr합 8.0
10946 번째 wr합 11.0
10947

11368 번째 wr합 6.0
11369 번째 wr합 9.0
11370 번째 wr합 11.0
11371 번째 wr합 10.0
11372 번째 wr합 6.0
11373 번째 wr합 13.0
11374 번째 wr합 6.0
11375 번째 wr합 8.0
11376 번째 wr합 10.0
11377 번째 wr합 9.0
11378 번째 wr합 10.0
11379 번째 wr합 9.0
11380 번째 wr합 13.0
11381 번째 wr합 10.0
11382 번째 wr합 8.0
11383 번째 wr합 3.0


In [12]:
# drop_val2 변수 제거
drop_val2 = [ 'wr_01', 'wr_02', 'wr_03', 'wr_04','wr_05', 'wr_06', 'wr_07', 'wr_08',
             'wr_09', 'wr_10', 'wr_11', 'wr_12','wr_13']

train = train.drop (drop_val2, axis=1)
test = test.drop (drop_val2, axis=1)

In [45]:
#train data에서 feature와 label값 분리
train_x =train.drop('voted', axis=1)
train_y = train['voted']

print(train_x.shape)
print(train_y.shape)
print(test.shape)


(45532, 19)
(45532,)
(11383, 19)


In [49]:
#LGBClassifier로 Modeling
model = lgbm.LGBMClassifier(n_estimators=500)
model.fit(train_x, train_y, early_stopping_rounds=150, eval_metric='logloss', eval_set=evals, verbose=False)
pred_y=model.predict(test)
print(pred_y.shape)
pred_y

In [58]:
#submission
submission['voted']=pred_y
submission.to_csv('./sample_submission.csv')